<img src="https://github.com/christopherhuntley/BUAN6510/blob/master/img/Dolan.png?raw=true" width="180px" align="right">

# **BUAN 6510**
# **Lesson 10: Dimensional Data Warehouse Design** 
_Data Driven Analytics_

## **Learning Objectives**
### **Theory / Be able to explain ...**
- 

### **Skills / Know how to ...**
- 

--------
## **LESSON 10 HIGHLIGHTS**

In [1]:
#@title Run this cell if video does not appear
%%html
<div style="max-width:1000px">
  <div style="position: relative;padding-bottom: 56.25%;height: 0;">
    <iframe style="position: absolute;top: 0;left: 0;width: 100%;height: 100%;" rel="0" modestbranding="1"  src="https://www.youtube.com/embed/ZSVRiOfodDY" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
  </div>
</div>

## **BIG PICTURE: The Holy Grail of Data Warehousing**

Data Warehousing as a universal data repository has been a dream since Ralph Kimball first coined the term in the 1980s. Analysts had by that point built up quite a repertoire of models for just about any kind of analysis. They could create classification and regression trees (decision trees, random forests, etc.). They could do linear, nonlinear, kernel, and logistic regression with large datasets. They could solve optimization problems with thousands of variables and tens of thousands of constraints. Even the neural network models at the core of the lastest and great deep learning techniques were pretty mature by 1994.  

What was missing was data! Well, sort of. What we had was lots of transactional data locked up in siloed mainframe systems. Online banking (via ATMs) had been a reality since the 1970s, credit card transactions since the 1960s, and the air traffic control system since the 1950s. These systems were great at capturing event data, one event at a time. They could even handle bulk transactions and reporting, if you could wait for the job to run overnight. 

However, analytical data is not the same as transactional data. Among the differences: 
- **Temporal Scope:** Emphasis on historical data rather than real-time operations
- **Diversity:** Drawing on multiple sources instead of a single transaction system
- **Data Quality:** "good enough to run the company" was not good enough for analytical modeling
- **Performance Tradeoffs:** Focus on computational speed (on desktop computers) instead of data throughput (on shared mainframes)
- **Access:** Providing analysts with direct access to transaction systems is both insecure and expensive. 

Even back then the ultimate solutions were known, though not remotely close to being available. Consider, for example, this figure from Kimball's seminal book *The Data Warehouse Lifecycle Toolkit*, published in 1998 and based on original work from the mid-1980s.

![Kimball's Data Warehouse](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L10_Kimball_Data_Warehouse_Elements.png)

All of the elements of a modern data pipeline are there. They even articulated the steps of the ETL process in detail. It was all there ... to be realized *someday*. 

Someday is now. With commodity data storage, ample computing power, ready-made software for just about any kind of modeling, and the analytical results to attract attention from management, data infrastructure is finally seen as what it should have been all along: a critical resource upon which the company relies to make it stand out against the competition. 

That's what the vendors tell us, anyway. 

In this lesson we will explore the **Dimensional Data Warehouse** model first proposed by Kimball et al. all those years ago. We will also consider how its mass adoption has influenced the SQL standard in recent years, with the addition of extensions like  **arrays**, **structs**, and  **partitions** that relax fundamental assumptions of the relational data model. 

---
## **The Star Schema Pattern**
### **Dimension Tables**
### **Fact Tables**
### **Snowflakes and Galaxies**
 


---
## **Rollups and Drilldowns**

---
## **NBA LineupFacts Data Mart**









---
## **PRO TIPS: How to Handle Longitudinal Dimensions**





---
## **SQL AND BEYOND: Arrays, Structs, and Partitions**

---
## **Congratulations! You've made it to the end of Lesson 10.**

Next week we will consider alternative data models that can improve flexibility and performance. 



## **On your way out ... Be sure to save your work**.
In Google Drive, drag this notebook file into your `BUAN6510` folder so you can find it next time.